# Rapport

ETAPES:

a)Partie analyse lineaire:

    1-discrétisation de la structure 
    2-création des matrices K  et F
    3-mise en place des conditions aux limites
    4-resolution de K*U=F => obtention de U

b)Partie analyse du flambage 

    1-construction de la matrice G
    2-resolution de l'equation (K-lambda*G)*U=0 grace à la méthode puissance itérée 
    => obtention de lambda
    3-Vérification de notre coefficient lambda

Dans un premier temps, on a créé un modèle de notre structure en la discrétisant.

Nous avons par un cas simple, soit une poutre.
    
Nous avons décomposé une poutre en $n_{e}$ segments. Elle est constituée d'**éléments**, separés par $n$ **noeuds** avec pour coordonnées $\{(x_I,y_I)\}_{I=1}^{n}$.

Nous avons ensuite appliqué cette discrétisation à une structure plus complexe, le portique.